In [ ]:
#Applying Agumentations to the balanced data
from audiomentations import TimeStretch, Shift

transform = TimeStretch(
    min_rate=0.8,
    max_rate=1.25,
    # leave_length_unchanged=True,
    p=0.5
)
transform1= Shift(min_fraction=-0.2,
                  max_fraction=0.3,
                  rollover=True,
                  p=0.5)

In [ ]:
import os
import glob
import librosa
import soundfile as sf
import numpy as np
import pandas as pd
from sklearn.utils import resample

input_dir = "data"  # Directory containing your audio recordings
output_dir = "balanced_data_1"  # Directory to save the balanced dataset
os.makedirs(output_dir, exist_ok=True)

metadata_path = "data/metadata.csv"

# Load metadata.csv file
metadata = pd.read_csv(metadata_path)

# Define target number of instances per class
target_instances = 20

# Dictionary to store instances for each class
class_data = {}

# Iterate over each row in the metadata
for index, row in metadata.iterrows():
    file_path = os.path.join( row['path'])
    class_label = row['label']

    # Load audio
    audio, sr = librosa.load(file_path, sr=None)

    # Append audio and class label to class_data dictionary
    if class_label not in class_data:
        class_data[class_label] = []
    class_data[class_label].append(audio)


In [ ]:

# Perform class-wise sampling and data augmentation
for class_label, instances in class_data.items():
    # Determine the number of instances in the class
    num_instances = len(instances)

    # Check if class needs balancing
    if num_instances < target_instances:
        # Class needs oversampling
        num_samples = target_instances - num_instances

        # Randomly oversample instances from the class
        oversampled_instances = resample(instances, n_samples=num_samples, random_state=42)

        # Augment oversampled instances (e.g., pitch shifting, time stretching)
        augmented_instances = []
        for audio in oversampled_instances:
            # Apply augmentation techniques
            augmented_audio = transform(audio, sample_rate=44100)
            augmented_audio = transform1(augmented_audio, sample_rate=44100)
            
            # Append augmented instance to the list
            augmented_instances.append(augmented_audio)

        # Combine original and augmented instances
        balanced_instances = instances + augmented_instances

    # elif num_instances > target_instances:
    #     # Class needs undersampling
    #     balanced_instances = resample(instances, n_samples=target_instances, random_state=42)

    else:
        # Class already balanced
        balanced_instances = instances

    # Save balanced instances to the output directory
    # class_dir = os.path.join(output_dir, str(class_label))
    # os.makedirs(class_dir, exist_ok=True)

    for i, audio in enumerate(balanced_instances):
        # Generate a unique filename for each instance
        filename = f"{class_label}_{i}.wav"

        # Save the audio to the output directory
        output_path = os.path.join(output_dir,filename)
        sf.write(output_path, audio, sr)

# Creating the Train Valid split with a ratio of 80:20

In [ ]:
import os
import random
import shutil

# Set the path to your original folder containing the .wav files
original_folder = 'balanced_data_1'

# Set the paths to the train and val folders where the files will be split
train_folder = 'balanced_data_1/train'
val_folder = 'balanced_data_1/val'

# Create the train and val folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Set the ratio for train and val (80:20)
train_ratio = 0.8
val_ratio = 0.2

# Get a list of all the .wav files in the original folder
wav_files = [file for file in os.listdir(original_folder) if file.endswith('.wav')]

# Shuffle the list of files randomly
random.shuffle(wav_files)

# Split the files into train and val sets based on the ratios
train_files = wav_files[:int(len(wav_files) * train_ratio)]
val_files = wav_files[int(len(wav_files) * train_ratio):]

# Move the train files to the train folder
for file in train_files:
    shutil.move(os.path.join(original_folder, file), os.path.join(train_folder, file))

# Move the val files to the val folder
for file in val_files:
    shutil.move(os.path.join(original_folder, file), os.path.join(val_folder, file))


# Preparing the metadata for the dataset

In [ ]:
import os
import csv

# Set the directory path containing the audio files
directory = 'balanced_data_1/train'

# Set the path for the metadata.csv file
csv_file = 'balanced_data_1/train/metadata.csv'

# Open the metadata.csv file in write mode
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['file_name', 'label', 'path'])

    # Iterate over the audio files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.wav'):  # Modify the file extension if needed
            file_path = 'train'
            label = filename.split('_')[0]  # Extract the first word as the label

            # Write the file_name, label, and path to the metadata.csv file
            writer.writerow([filename, label, file_path])


In [ ]:
import os
import csv

# Set the directory path containing the audio files
directory = 'balanced_data_1/val'

# Set the path for the metadata.csv file
csv_file = 'balanced_data_1/val/metadata.csv'

# Open the metadata.csv file in write mode
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['file_name', 'label', 'path'])

    # Iterate over the audio files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.wav'):  # Modify the file extension if needed
            file_path = 'val'
            label = filename.split('_')[0]  # Extract the first word as the label

            # Write the file_name, label, and path to the metadata.csv file
            writer.writerow([filename, label, file_path])


In [ ]:
import pandas as pd

# Set the paths to the two CSV files
csv_file1 = 'balanced_data_1/train/metadata.csv'
csv_file2 = 'balanced_data_1/val/metadata.csv'

# Read the CSV files into pandas DataFrames
df1 = pd.read_csv(csv_file1)
df2 = pd.read_csv(csv_file2)

# Append the two DataFrames together
combined_df = df1.append(df2, ignore_index=True)

# Set the path for the combined CSV file
combined_csv_file = 'balanced_data_1/metadata.csv'

# Write the combined DataFrame to a new CSV file
combined_df.to_csv(combined_csv_file, index=False)
